In [1]:
# !pip install netCDF4
import netCDF4 as nc
import xarray as xr
import pandas as pd
import time
from ftplib import FTP
import os
from datetime import datetime
import csv

In [2]:
# variable_dict = {
#     "aerosol_optical_thickness_over_water":"__GLOB_RESOLUTION_AVW-MODVIR_T865_DAY_00",
#     "aerosol_optical_thickness_over_water+land":"__GLOB_RESOLUTION_AVW-MODVIR_T550_DAY_00",
#     "cloud_fraction ":"__GLOB_RESOLUTION_AV-MODVIR_CF_DAY_00",
#     "depth_of_the_bottom_of_the_euphotic_layer":"__GLOB_RESOLUTION_AVW-MODVIROLAVJ1OLB_ZEU_DAY_00",
#     "secchi_disk_depth":"__GLOB_RESOLUTION_AVW-MODVIR_ZSD-DORON_DAY",
#     "heated_layer_depth":"__GLOB_RESOLUTION_AVW-MODVIR*_ZHL_DAY_00",
#     "particulate_backscattering_coefficient":"__GLOB_RESOLUTION_GSM-MODVIR_BBP_DAY_00",
#     "coloured_dissolved_detrital_organic_materials_absorption_coefficient":"__GLOB_RESOLUTION_GSM-MODVIR_CDM_DAY_00",
#     "diffuse_attenuation_coefficient":"__GLOB_RESOLUTION_AVW-MODVIR_KD490-LEE_DAY_00",
#     "relative_excess_of_radiance":"__GLOB_RESOLUTION_AVW-MODVIR_EL555_DAY_00",
#     "normalised_remote_sensing_reflectance":"__GLOB_RESOLUTION_AVW-MODVIR_NRRS490_DAY_00",
#     "normalised_fluorescence_line_height":"__GLOB_RESOLUTION_AV-MOD_NFLH_DAY_00",
#     "particulate_organic_carbon":"__GLOB_RESOLUTION_AVW-MODVIR_POC_DAY_00",
#     "particulate_inorganic_carbon":"__GLOB_RESOLUTION_AVW-MODVIR_PIC_DAY_00",
#     "inorganic_suspended_particulate_matter_concentration":"__GLOB_RESOLUTION_AVW-MODVIROLAVJ1OLB_SPM-OC5_DAY_00",
#     "photosynthetically_available_radiation":"__GLOB_RESOLUTION_AVW-MODVIR_PAR_DAY_00"
    
# }

# variable dictionary
reader = csv.DictReader(open('C:/Users/javi2/Documents/CD_aplicada_1/COBI/other/variable_dict.csv'))
variable_dict = {}

for row in reader:
    variable_dict[row['variable']] = row['file_format']

# # Ver qué pez con las variables de entorno
# directory_to = os.environ.get("PATH_SIMAR")

# credentials = [os.environ.get("GLOBCOLOUR_USER"), os.environ.get('GLOBCOLOUR_PWD')]
                                         
# Ver qué pez con las variables de entorno
directory_to = "C:\\Users\\javi2\\Documents\\CD_aplicada_1\\COBI\\data\\simar\\raw\\"

credentials = ['ftp_gc_JOrcazas_Leal', 'JOrcazas_Leal_6463']

In [3]:
def download_raw(variable, directory_to, resolution, credentials, variable_dict, year_from):
    
    years = list(range(year_from, datetime.now().year+1))
    months = ["01","02","03","04","05","06","07","08","09","10","11","12"]
    days = ["01","02","03","04","05","06","07","08","09","10","11","12", "13","14","15",
            "16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31"]

    user = credentials[0] 
    password = credentials[1] 

    directory_path = directory_to + variable
    if not os.path.isdir(directory_path):
        os.mkdir(directory_path)
    
    os.chdir(directory_path) #changes the active dir - this is where downloaded files will be saved to
    
    
    filematch = 'L3m_*'+variable_dict[variable].replace("RESOLUTION", resolution)+'.nc' # a match for any file in this case, can be changed or left for user to input
    
    not_loaded = []
    
    for year in years:
        for month in months:
            for day in days:
                try:                    
                    download_dir ="GLOB/merged/day/"+str(year)+"/"+month+"/"+day+"/" #dir i want to download files from, can be changed or left for user input
                    ftp = FTP("ftp.hermes.acri.fr")
                    ftp.login(user,password)
                    
                    ftp.cwd(download_dir)
                    for filename in ftp.nlst(filematch): # Loop - looking for matching files
                        if not os.path.exists(filename):
                            fhandle = open(filename, 'wb')
        #                     print('Getting ' + filename) #for confort sake, shows the file that's being retrieved
                            ftp.retrbinary('RETR ' + filename, fhandle.write)
                            fhandle.close()
                        
                    ftp.quit()
                except Exception as e:
                    not_loaded.append((year, month, day, e))
                    print("error in variable", variable,":", e, str(year)+str(month)+str(day))

    return not_loaded

In [ ]:
results = {}
for v in variable_dict.keys():
    results[v] = download_raw(v, directory_to, "100", credentials, variable_dict, 2017)

Volver a correr estas o revisar: 

"depth_of_the_bottom_of_the_euphotic_layer":"__GLOB_RESOLUTION_AVW-MODVIROLAVJ1OLB_ZEU_DAY_00",

"secchi_disk_depth":"__GLOB_RESOLUTION_AVW-MODVIR_ZSD-DORON_DAY",

"heated_layer_depth":"__GLOB_RESOLUTION_AVW-MODVIROLAVJ1OLB_ZHL_DAY_00",

"inorganic_suspended_particulate_matter_concentration":"__GLOB_RESOLUTION_AVW-MODVIROLAVJ1OLB_SPM-OC5_DAY_00"

    

In [5]:
import requests
import os
import pandas as pd
import time
import glob
from datetime import datetime

def read_cicese_data(place):
    columns=["anio","mes","dia","hora","minuto","segundo",
             "id_estacion","voltaje_sistema","nivel_mar_leveltrol","nivel_mar_burbujeador",
             "sw_1","sw_2","temperatura_agua","nivel_mar_ott_rsl", "radiacion_solar",
             "direccion_viento", "magnitud_viento", "temperatura_aire","humedad_relativa",
             "presion_atmosferica","precipitacion","voltaje_estacion_met","nivel_mar_sutron"]
    # Set the directory path
    dir_path = 'data/'+place
    
    # Get a list of all .dat files in the directory
    dat_files = glob.glob(os.path.join(dir_path, "*.dat"))

    # Initialize an empty list to store the dataframes
    dfs = []

    # Loop through each file and read it into a dataframe
    for file in dat_files:
        df = pd.read_csv(file, lineterminator='\n', delim_whitespace=True, header=None)
        dfs.append(df)

    # Concatenate all the dataframes into a single dataframe
    result_df = pd.concat(dfs, axis=0, ignore_index=True)
    
    # Rename df columns with the ones defined before
    dict_columns = {}
    for col, i in zip(columns, range(len(columns))):
        dict_columns[i] = col
    dict_columns
    result_df = result_df.rename(columns=dict_columns)

    
    return result_df


df = read_cicese_data("isla_cedros")


Index(['anio', 'mes', 'dia', 'hora', 'minuto', 'segundo', 'id_estacion',
       'voltaje_sistema', 'nivel_mar_leveltrol', 'nivel_mar_burbujeador',
       'sw_1', 'sw_2', 'temperatura_agua', 'nivel_mar_ott_rsl',
       'radiacion_solar', 'direccion_viento', 'magnitud_viento',
       'temperatura_aire', 'humedad_relativa', 'presion_atmosferica',
       'precipitacion', 'voltaje_estacion_met', 'nivel_mar_sutron'],
      dtype='object')

In [6]:
df.columns

Index(['anio', 'mes', 'dia', 'hora', 'minuto', 'segundo', 'id_estacion',
       'voltaje_sistema', 'nivel_mar_leveltrol', 'nivel_mar_burbujeador',
       'sw_1', 'sw_2', 'temperatura_agua', 'nivel_mar_ott_rsl',
       'radiacion_solar', 'direccion_viento', 'magnitud_viento',
       'temperatura_aire', 'humedad_relativa', 'presion_atmosferica',
       'precipitacion', 'voltaje_estacion_met', 'nivel_mar_sutron'],
      dtype='object')